# <center> </center>
# <center> **Computer Vision** </center>
# <center> **Portfolio Exam 1**</center>
# <center>**Object Detection for Wildlife Conservation - Detecting Waterfowl in UAV Thermal Imagery**</center>


**Submitted by:**
****

*   **Riya Biju - 10000742**
*   **Harsha Sathish - 10001000**
*   **Harshith Babu Prakash Babu - 10001191**


**Assignment 1: Object Detection for Wildlife Conservation - Detecting
Waterfowl in UAV Thermal Imagery (15 Points)**

Unmanned Aerial Vehicles (UAVs, or drones) are increasingly used in wildlife conservation
for non-invasive monitoring of animal populations. They enable researchers to capture aerial
imagery that supports automated species detection and population estimation.
In this assignment, you will develop and evaluate an object detection model for identifying
waterfowl (aquatic birds) in aerial images. You will use the UAV-derived Waterfowl Thermal
Imagery Dataset1
, which includes both thermal and RGB images of wetlands with annotated
bounding boxes around individual birds.
1 https://data.mendeley.com/datasets/46k66mz9sz/2

# **1. Importing all necessary python modules**

In [ ]:
import os
import random
import shutil
from pathlib import Path
import numpy as np
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from ultralytics import YOLO
import torch
import pandas as pd

# Set random seeds for reproducibility
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

print("✓ Libraries imported successfully")

✓ Libraries imported successfully


# **2. Configuration and Repository Structure**

In [14]:
DATA_ROOT = '/Users/harshasathish/Desktop/MAI/ComputerVision/Portfolio1_new'
OUTPUT_ROOT = 'output'

# Training hyperparameters
EPOCHS = 100
IMG_SIZE = 640
BATCH_SIZE = 8
TRAIN_RATIO = 0.7
VAL_RATIO = 0.15
TEST_RATIO = 0.15

print(f"✓ Configuration set")
print(f"  Data root: {DATA_ROOT}")
print(f"  Output root: {OUTPUT_ROOT}")

data_root = Path(DATA_ROOT)
output_root = Path(OUTPUT_ROOT)
output_root.mkdir(exist_ok=True)

# Create directory structure
yolo_root = output_root / 'yolo_dataset'
model_dir = output_root / 'models'
results_dir = output_root / 'results'

print("\n✓ Cleaning old output directories...")
# Clean and create output directories
for d in [yolo_root, model_dir, results_dir]:
    if d.exists():
        print(f"  Removing {d}")
        shutil.rmtree(d)
    d.mkdir(parents=True, exist_ok=True)

print(f"✓ Directory structure initialized")

✓ Configuration set
  Data root: /Users/harshasathish/Desktop/MAI/ComputerVision/Portfolio1_new
  Output root: output

✓ Cleaning old output directories...
✓ Directory structure initialized


# **3. Load and validate Data**

In [ ]:
print("\n" + "="*60)
print("STEP 1: Dataset Preparation")
print("="*60)

# Define paths
positive_img_dir = data_root / 'thermal_Images' / '01_Positive_img'
negative_img_dir = data_root / 'thermal_Images' / '03_Negative'
annotation_dir = data_root / 'thermal_Images' / '02_Positive_annotation'
csv_path = annotation_dir / 'Bounding Box Label.csv'

# Check if paths exist
if not positive_img_dir.exists():
    raise FileNotFoundError(f"Image directory not found: {positive_img_dir}")
if not negative_img_dir.exists():
    raise FileNotFoundError(f"Negative Image directory not found: {negative_img_dir}")
if not csv_path.exists():
    raise FileNotFoundError(f"CSV file not found: {csv_path}")

# Read CSV annotations
print(f"\n✓ Reading annotations from CSV...")
df = pd.read_csv(csv_path)
print(f"✓ Found {len(df)} annotations in CSV")

# Get all .tif image files (Positive AND Negative)
positive_files = list(positive_img_dir.glob('*.tif'))
negative_files = list(negative_img_dir.glob('*.tif'))

print(f"\n✓ Found {len(positive_files)} positive thermal images")
print(f"✓ Found {len(negative_files)} negative thermal images")

# Combine all images
all_image_files = positive_files + negative_files
print(f"✓ Total dataset size: {len(all_image_files)} images")


STEP 1: Dataset Preparation

✓ Reading annotations from CSV...
✓ Found 8975 annotations in CSV
✓ CSV columns: ['imageFilename', 'x(column)', 'y(row)', 'width', 'height']

✓ Found 355 thermal images (.tif)


# **4. Create annotations dictionary**

In [16]:
annotations_dict = {}
# Group annotations by image filename
for _, row in df.iterrows():
    img_name = row['imageFilename']
    if img_name not in annotations_dict:
        annotations_dict[img_name] = []
    annotations_dict[img_name].append(row)

print(f"✓ Found annotations for {len(annotations_dict)} unique images")

# Filter images that have annotations
valid_images = []
for img_path in image_files:
    if img_path.name in annotations_dict:
        valid_images.append(img_path)

print(f"✓ Found {len(valid_images)} images with annotations")

# Check if we have matching data
if len(valid_images) == 0:
    print("\n⚠ WARNING: No matching images found!")
    print("Sample annotation filenames:")
    for name in list(annotations_dict.keys())[:5]:
        print(f"  - {name}")
    print("\nSample image filenames:")
    for img in image_files[:5]:
        print(f"  - {img.name}")
    raise ValueError("No images match the annotations in the CSV")

✓ Found annotations for 355 unique images
✓ Found 355 images with annotations


# **5. Split Dataset**

In [17]:
# First split: train vs temp (val+test)
train_imgs, temp_imgs = train_test_split(
    valid_images,
    test_size=(VAL_RATIO + TEST_RATIO),
    random_state=42
)

# Second split: val vs test
val_imgs, test_imgs = train_test_split(
    temp_imgs,
    test_size=TEST_RATIO/(VAL_RATIO + TEST_RATIO),
    random_state=42
)

print(f"\n✓ Dataset split:")
print(f"  Training:   {len(train_imgs)} images ({TRAIN_RATIO*100:.0f}%)")
print(f"  Validation: {len(val_imgs)} images ({VAL_RATIO*100:.0f}%)")
print(f"  Test:       {len(test_imgs)} images ({TEST_RATIO*100:.0f}%)")

splits = {
    'train': train_imgs,
    'val': val_imgs,
    'test': test_imgs
}


✓ Dataset split:
  Training:   248 images (70%)
  Validation: 53 images (15%)
  Test:       54 images (15%)


# **6. Process and convert Dataset**

In [18]:
print("\n✓ Clearing old dataset directories...")
# Remove old split directories
for split_name in splits:
    split_dir = yolo_root / split_name
    if split_dir.exists():
        print(f"  Removing {split_dir}")
        shutil.rmtree(split_dir)

# Process each split
for split_name, img_list in splits.items():
    # Create directories
    img_dir = yolo_root / split_name / 'images'
    label_dir = yolo_root / split_name / 'labels'
    img_dir.mkdir(parents=True, exist_ok=True)
    label_dir.mkdir(parents=True, exist_ok=True)

    print(f"\n✓ Processing {split_name} set...")

    # Process each image
    for img_path in img_list:
        # Read thermal image
        img = cv2.imread(str(img_path), cv2.IMREAD_GRAYSCALE)

        if img is None:
            print(f"  ⚠ Warning: Could not read {img_path.name}")
            continue

        img_height, img_width = img.shape

        # Convert to RGB (3-channel)
        img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)

        # Save converted image
        dest_img = img_dir / img_path.name
        cv2.imwrite(str(dest_img), img_rgb)

        # Get annotations for this image
        annotations = annotations_dict[img_path.name]

        yolo_annotations = []
        invalid_count = 0

        # Convert each annotation to YOLO format
        for ann in annotations:
            # Get bounding box in pixel coordinates from CSV
            x1_px = float(ann['x(column)'])
            y1_px = float(ann['y(row)'])
            w_px = float(ann['width'])
            h_px = float(ann['height'])

            # Skip boxes with zero or negative dimensions
            if w_px <= 0 or h_px <= 0:
                invalid_count += 1
                continue

            # Calculate x2, y2 in pixels
            x2_px = x1_px + w_px
            y2_px = y1_px + h_px

            # Clip pixel coordinates to image boundaries
            x1_clipped = max(0.0, x1_px)
            y1_clipped = max(0.0, y1_px)
            x2_clipped = min(float(img_width), x2_px)
            y2_clipped = min(float(img_height), y2_px)

            # Recalculate width/height from clipped coordinates
            w_clipped = x2_clipped - x1_clipped
            h_clipped = y2_clipped - y1_clipped

            # Skip if box is now invalid
            if w_clipped <= 0.001 or h_clipped <= 0.001:
                invalid_count += 1
                continue

            # Convert to YOLO format (normalized coordinates)
            x_center = (x1_clipped + w_clipped / 2) / img_width
            y_center = (y1_clipped + h_clipped / 2) / img_height
            norm_width = w_clipped / img_width
            norm_height = h_clipped / img_height

            # Class 0 for waterfowl
            yolo_annotations.append(f"0 {x_center:.6f} {y_center:.6f} {norm_width:.6f} {norm_height:.6f}")

        # Log filtered boxes for train set
        if invalid_count > 0 and split_name == 'train':
            print(f"  ⚠ {img_path.name}: filtered {invalid_count} invalid boxes")

        # Save annotations if valid
        if len(yolo_annotations) > 0:
            dest_ann = label_dir / f"{img_path.stem}.txt"
            with open(dest_ann, 'w') as f:
                f.write('\n'.join(yolo_annotations))
        else:
            # Skip this image if no valid annotations
            if dest_img.exists():
                dest_img.unlink()
            if split_name != 'test':
                print(f"  ⚠ {img_path.name}: skipped (no valid annotations)")

    print(f"  Processed {len(img_list)} images")




✓ Clearing old dataset directories...

✓ Processing train set...
  Processed 248 images

✓ Processing val set...
  Processed 53 images

✓ Processing test set...
  Processed 54 images


# **7. YOLO Configuration File**

In [19]:
yaml_content = f"""# Waterfowl Detection Dataset
path: {yolo_root.absolute()}
train: train/images
val: val/images
test: test/images

# Classes
nc: 1
names: ['waterfowl']
"""

yaml_path = yolo_root / 'data.yaml'
with open(yaml_path, 'w') as f:
    f.write(yaml_content)

print(f"\n✓ Dataset prepared successfully!")
print(f"  YOLO format saved to: {yolo_root}")
print(f"  Configuration: {yaml_path}")


✓ Dataset prepared successfully!
  YOLO format saved to: output/yolo_dataset
  Configuration: output/yolo_dataset/data.yaml


# **8. Validating Dataset**

In [20]:
print(f"\n✓ Validating dataset...")

overall_issues = 0

# Check each split
for split in ['train', 'val', 'test']:
    img_dir = yolo_root / split / 'images'
    label_dir = yolo_root / split / 'labels'

    if not img_dir.exists() or not label_dir.exists():
        print(f"  Skipping {split}: directory not found.")
        continue

    # Get stems of all files
    img_files = {p.stem for p in img_dir.glob('*.*') if p.suffix in ['.tif', '.jpg', '.png']}
    label_files = {p.stem for p in label_dir.glob('*.txt')}

    # Check for orphan labels
    orphan_labels = label_files - img_files
    if orphan_labels:
        print(f"  ⚠ {split}: Found {len(orphan_labels)} orphan label files (no matching image).")
        for orphan in list(orphan_labels)[:5]:
            print(f"    - {orphan}.txt")
        overall_issues += len(orphan_labels)

    # Check for images without labels
    background_images = img_files - label_files
    if background_images:
        print(f"  ℹ {split}: Found {len(background_images)} images with no labels (treated as backgrounds).")

    # Validate label content
    issues_in_split = 0
    total_boxes = 0
    empty_files = 0

    # Only check labels that have matching images
    for label_stem in (label_files & img_files):
        label_file = label_dir / f"{label_stem}.txt"

        # Check for empty files
        if label_file.stat().st_size == 0:
            print(f"  ⚠ {label_file.name}: FILE IS EMPTY (0 bytes).")
            issues_in_split += 1
            empty_files += 1
            continue

        with open(label_file, 'r') as f:
            lines = f.readlines()
            if not lines:
                print(f"  ⚠ {label_file.name}: FILE IS EMPTY (no lines).")
                issues_in_split += 1
                empty_files += 1
                continue

            # Validate each line
            for line_num, line in enumerate(lines, 1):
                line = line.strip()
                if not line:
                    continue

                parts = line.split()
                if len(parts) != 5:
                    print(f"  ⚠ {label_file.name} line {line_num}: wrong format (expected 5 parts, got {len(parts)})")
                    issues_in_split += 1
                    continue

                try:
                    cls, x, y, w, h = map(float, parts)
                    total_boxes += 1

                    # Check if values are in valid range
                    if not (0 <= x <= 1 and 0 <= y <= 1 and 0 < w <= 1 and 0 < h <= 1):
                        print(f"  ⚠ {label_file.name} line {line_num}: values out of range")
                        print(f"    x={x:.3f}, y={y:.3f}, w={w:.3f}, h={h:.3f}")
                        issues_in_split += 1
                except ValueError:
                    print(f"  ⚠ {label_file.name} line {line_num}: invalid values (not numbers)")
                    issues_in_split += 1

    print(f"  {split}: {len(label_files)} labels, {len(img_files)} images.")
    print(f"    - {total_boxes} total boxes (from matching files).")
    print(f"    - {empty_files} empty label files.")
    print(f"    - {issues_in_split - empty_files} line/format issues.")
    overall_issues += issues_in_split

# Final validation result
if overall_issues > 0:
    print(f"\n WARNING: Found {overall_issues} total annotation issues. Training may fail.")
else:
    print(f"\n All annotations are valid!")


✓ Validating dataset...
  train: 248 labels, 248 images.
    - 6029 total boxes (from matching files).
    - 0 empty label files.
    - 0 line/format issues.
  val: 53 labels, 53 images.
    - 1295 total boxes (from matching files).
    - 0 empty label files.
    - 0 line/format issues.
  test: 54 labels, 54 images.
    - 1651 total boxes (from matching files).
    - 0 empty label files.
    - 0 line/format issues.

 All annotations are valid!


# **9. Training YOLO**

In [21]:
print("\n" + "="*60)
print("STEP 2: Model Training")
print("="*60)

device = 'cpu'

# Load pretrained YOLOv8 model
print("\n✓ Loading YOLOv8n (nano) pretrained model...")
model = YOLO('yolov8n.pt')

print(f"\n✓ Starting training...")
print(f"  Epochs: {EPOCHS}")
print(f"  Image size: {IMG_SIZE}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Device: {device}")

# Train the model
results = model.train(
    data=str(yaml_path),
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    device=device,
    project=str(model_dir),
    name='waterfowl_detector',
    exist_ok=True,
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10,
    translate=0.1,
    scale=0.5,
    fliplr=0.5,
    flipud=0.0,
    mosaic=0.0,
)

# Save best model path
best_model_path = model_dir / 'waterfowl_detector' / 'weights' / 'best.pt'

print(f"\n✓ Training complete!")
print(f"  Best model saved to: {best_model_path}")



STEP 2: Model Training

✓ Loading YOLOv8n (nano) pretrained model...

✓ Starting training...
  Epochs: 100
  Image size: 640
  Batch size: 8
  Device: cpu
New https://pypi.org/project/ultralytics/8.3.228 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.227 🚀 Python-3.13.2 torch-2.9.0 CPU (Apple M3 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=output/yolo_dataset/data.yaml, degrees=10, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=100, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_

# **10. Model Evaluation**

In [22]:
print("\n" + "="*60)
print("STEP 3: Model Evaluation")
print("="*60)

# Load the best model
model = YOLO(str(best_model_path))
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# Run validation on test set
print("\n✓ Evaluating on test set...")

metrics = model.val(
    data=str(yaml_path),
    split='test',
    device=device
)

print("\n✓ Evaluation Results:")
print(f"  mAP@50:    {metrics.box.map50:.4f}")
print(f"  mAP@50-95: {metrics.box.map:.4f}")
print(f"  Precision: {metrics.box.mp:.4f}")
print(f"  Recall:    {metrics.box.mr:.4f}")



STEP 3: Model Evaluation

✓ Evaluating on test set...
Ultralytics 8.3.227 🚀 Python-3.13.2 torch-2.9.0 MPS (Apple M3 Pro)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.1±0.0 ms, read: 1062.2±272.5 MB/s, size: 343.1 KB)
val: Scanning /Users/harshasathish/Desktop/MAI/ComputerVision/Portfolio1_new/output/yolo_dataset/test/labels... 54 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 54/54 3.8Kit/s 0.0s
val: New cache created: /Users/harshasathish/Desktop/MAI/ComputerVision/Portfolio1_new/output/yolo_dataset/test/labels.cache
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100% ━━━━━━━━━━━━ 4/4 0.6it/s 6.6s2.3ss
                   all         54       1651      0.919      0.915      0.948      0.571
Speed: 24.1ms preprocess, 6.5ms inference, 0.0ms loss, 20.9ms postprocess per image
Results saved to /Users/harshasathish/Desktop/MAI/ComputerVision/Portfolio1_new/runs/detect/val7



# **11. Result Visualization**

In [23]:
print("\n" + "="*60)
print("STEP 4: Visualization")
print("="*60)

model = YOLO(str(best_model_path))
device = 'mps' if torch.backends.mps.is_available() else 'cpu'

# Get test images
test_img_dir = yolo_root / 'test' / 'images'
test_label_dir = yolo_root / 'test' / 'labels'
test_images = list(test_img_dir.glob('*.tif')) + list(test_img_dir.glob('*.jpg')) + list(test_img_dir.glob('*.png'))

# Categories for visualization
categories = {
    'true_positives': [],
    'false_negatives': [],
    'false_positives': []
}

print(f"\n✓ Analyzing {len(test_images)} test images...")

# Run inference and categorize results
for img_path in test_images:
    # Load image
    img = cv2.imread(str(img_path))
    if img is None:
        continue
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    # Get predictions
    results = model(img_path, device=device, verbose=False)
    predictions = results[0].boxes

    # Load ground truth
    label_path = test_label_dir / f"{img_path.stem}.txt"
    gt_boxes = []
    if label_path.exists():
        with open(label_path, 'r') as f:
            for line in f:
                parts = line.strip().split()
                if len(parts) == 5:
                    gt_boxes.append([float(x) for x in parts[1:]])

    # Categorize based on predictions and ground truth
    num_predictions = len(predictions)
    num_ground_truth = len(gt_boxes)

    if num_ground_truth > 0 and num_predictions > 0:
        categories['true_positives'].append((img_path, img_rgb, predictions, gt_boxes))
    elif num_ground_truth > 0 and num_predictions == 0:
        categories['false_negatives'].append((img_path, img_rgb, predictions, gt_boxes))
    elif num_ground_truth == 0 and num_predictions > 0:
        categories['false_positives'].append((img_path, img_rgb, predictions, gt_boxes))


STEP 4: Visualization

✓ Analyzing 54 test images...


# **12. Data Visualization Plots**

In [24]:
num_examples = 3

# Visualize each category
for category_name, examples in categories.items():
    if len(examples) == 0:
        print(f"\n! No {category_name} found")
        continue

    # Select random examples
    selected = random.sample(examples, min(num_examples, len(examples)))

    print(f"\n✓ Visualizing {category_name}: {len(selected)} examples")

    fig, axes = plt.subplots(1, len(selected), figsize=(5*len(selected), 5))
    if len(selected) == 1:
        axes = [axes]

    # Plot each example
    for idx, (img_path, img_rgb, predictions, gt_boxes) in enumerate(selected):
        ax = axes[idx]
        ax.imshow(img_rgb)

        h, w = img_rgb.shape[:2]

        # Draw ground truth boxes (green)
        for box in gt_boxes:
            x_center, y_center, width, height = box
            x1 = int((x_center - width/2) * w)
            y1 = int((y_center - height/2) * h)
            x2 = int((x_center + width/2) * w)
            y2 = int((y_center + height/2) * h)

            rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                fill=False, color='green', linewidth=2)
            ax.add_patch(rect)

        # Draw predictions (red) - WITHOUT labels
        if len(predictions) > 0:
            for box in predictions.xyxy:
                x1, y1, x2, y2 = box.cpu().numpy()
                rect = plt.Rectangle((x1, y1), x2-x1, y2-y1,
                                    fill=False, color='red', linewidth=2)
                ax.add_patch(rect)

        ax.set_title(f"{img_path.name}", fontsize=10)
        ax.axis('off')

    plt.tight_layout()
    save_path = results_dir / f'{category_name}.png'
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    print(f"  Saved to: {save_path}")
    plt.close()

print(f"\n✓ Visualization complete!")

print("\n" + "="*60)
print("PIPELINE COMPLETE!")
print("="*60)
print(f"\nAll outputs saved to: {OUTPUT_ROOT}/")


✓ Visualizing true_positives: 3 examples
  Saved to: output/results/true_positives.png

! No false_negatives found

! No false_positives found

✓ Visualization complete!

PIPELINE COMPLETE!

All outputs saved to: output/
